In [ ]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--conf spark.ui.port=4040 --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.0,com.datastax.spark:spark-cassandra-connector_2.11:2.0.0-M3 pyspark-shell'
import time

In [ ]:
import pyspark_cassandra

from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import time

topic_name="seek-jobs"
cass_host="localhost"
cass_port="9042"
cass_keyspace="client"
cass_table_name="info"

def save_to_cassandra(df,epoch):
    print(epoch)
    df.write.format("org.apache.spark.sql.cassandra") \
    .mode("append")\
    .options(table=cass_table_name,keyspace=cass_keyspace).save()
if __name__ == '__main__':
    print("spark started")
    spark = SparkSession.builder.appName('hello').master('local[*]') \
     .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.0")\
      .getOrCreate()
          
      
    df = spark.readStream.format("kafka").option("kafka.bootstrap.servers","localhost:9092") \
    .option("subscribe",topic_name).option("startingOffsets","latest").load()
    #print(stream_df.isStreaming) 
    
    df.printSchema()
    
    stream_df1 = df.selectExpr("CAST(value AS STRING)","timestamp")
    stream_schema = StructType().add("description",StringType()).add("title",StringType()) \
    .add("salary",StringType()).add("company",StringType()).add("location",StringType()) \
    .add("class",StringType()).add("days_before",StringType())

    stream_df2 = stream_df1.select(from_json(col("value"), stream_schema).alias("stream"),"timestamp")

    stream_df3=stream_df2.select("stream.*","timestamp")
    stream_df3.printSchema()
    stream_df4= stream_df3.groupBy("class").count()
    
    
    
    

    write_df = stream_df4.writeStream \
     .trigger(processingTime='15 seconds') \
     .outputMode("update")\
     .format("console").start()
     #.foreachBatch(save_to_cassandra)\
     
    

    write_df.awaitTermination()


spark started
root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

root
 |-- description: string (nullable = true)
 |-- title: string (nullable = true)
 |-- salary: string (nullable = true)
 |-- company: string (nullable = true)
 |-- location: string (nullable = true)
 |-- class: string (nullable = true)
 |-- days_before: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)

